# 可迭代的对象、迭代器和生成器

迭代器用于从集合中取出元素；生成器用于“凭空”生成元素。Python中所有集合均可以迭代，迭代器支持如下功能的正常运作：
1. for循环
2. 构建和扩展集合类型
3. 逐行遍历文本文件
4. 列表推导、字典推导和集合推导
5. 元组拆包
6. 调用函数时，使用*拆包实参

## 可迭代的序列

当解释器迭代对象时，会自动调用iter()函数以生成迭代器。具体来说，iter()函数会按照如下流程尝试获取一个迭代器：
1. 检查对象是否有__iter__方法。若有，则调用该函数并返回一个迭代器；否则进行第2步
2. 检查对象是否有__getitem__方法。若有，则创建一个迭代器，该迭代器尝试按照索引顺序获取元素（从0开始）；否则进行第3步
3. 抛出TypeError，并提示该对象不可迭代

从鸭子类型（duck typing）的角度来看：上述流程显然是符合鸭子类型的，只要对象实现了__iter__方法，或者__getitem__方法，该对象就可以迭代
从白鹅类型（goose typing）的角度来看：可迭代的对象更简单，但是更局限。只要实现了__iter__方法，就认为该对象是可迭代的。若仅实现了__getitem__方法，则无法通过issubclass的类型检查。

鉴于上述描述，若要在Python中检查一个对象是否是可迭代的，最好是直接使用iter()尝试构建迭代器，而不是使用issubclass检查该对象对应的类是否是abc.Iterable的子类。

## 可迭代对象与迭代器

**可迭代对象不是迭代器，可迭代对象可以通过iter()函数返回一个迭代器**。

for循环和while循环不同。for循环一个可迭代对象时，会自动生成一个迭代器用于迭代。但是while循环则不具有这样的功能，若想在while循环中迭代一个可迭代对象，开发者需要手动创建迭代器。

下述例子展示了for循环和while循环迭代一个可迭代对象时的区别。字节码清晰描述了这两种方式的区别。对于for循环，解释器使用字节码GET_ITER从可迭代对象生成了一个迭代器用于迭代；对于while循环，解释器使用字节码CALL_FUNCTION，尝试利用iter()函数从可迭代对象生成一个迭代器。

In [16]:
import dis

def for_iter(s):
    print("for循环迭代字符串s：")
    for char in s:
        print(char)

def while_iter(s):
    print("\n\nwhile循环迭代字符串s：")
    _it = iter(s)
    while True:
        try:
            print(next(_it))
        except StopIteration:
            del _it
            break

s = "ABC"
for_iter(s)
print("for循环字节码：")
print(dis.dis(for_iter))
while_iter(s)
print("while循环字节码：")
print(dis.dis(while_iter))

for循环迭代字符串s：
A
B
C
for循环字节码：
  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('for循环迭代字符串s：')
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 SETUP_LOOP              20 (to 30)
             10 LOAD_FAST                0 (s)
             12 GET_ITER
        >>   14 FOR_ITER                12 (to 28)
             16 STORE_FAST               1 (char)

  6          18 LOAD_GLOBAL              0 (print)
             20 LOAD_FAST                1 (char)
             22 CALL_FUNCTION            1
             24 POP_TOP
             26 JUMP_ABSOLUTE           14
        >>   28 POP_BLOCK
        >>   30 LOAD_CONST               0 (None)
             32 RETURN_VALUE
None


while循环迭代字符串s：
A
B
C
while循环字节码：
  9           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('\n\nwhile循环迭代字符串s：')
              4 CALL_FUNCTION            1
              6 POP_TOP

 10           8 LOAD_GLOBA

### 迭代器的本质

迭代器：迭代器是这样一种对象：一个实现了无参数__next__方法用于返回序列中下一个元素，若没有下一个元素则抛出StopIteration异常的对象。

内置的迭代器仅有两个方法：__next__和__iter__。而__iter__函数会返回迭代器对象自身。因此迭代器无法检查元素数量、无法检查是否有遗留元素并且也无法“复原”已经迭代过的元素。

### 可迭代对象不是迭代器

问题：一个对象是否可以既是可迭代对象同时也是迭代器？

假设一个对象是可迭代对象，那么该对象应当实现了__iter__方法；若该对象是迭代器，则该对象实现了__next__和__iter__，并且__iter__返回该对象自身。当使用iter()函数调用该对象时，__iter__会返回一个迭代器，此时__iter__会返回其自身。

设计模式中对迭代器模型有如下描述：
1. 迭代器可以访问一个聚合对象的内容而无需暴露它的内部表示
2. 迭代器支持对聚合对象的多种遍历
3. 迭代器为遍历不同的聚合结构提供一个统一的接口（支持多态）

上述第二点实际上表明应当能从一个对象中获取多个独立的迭代器。这些迭代器应当能够单独维护各自的状态。若一个对象是可迭代对象的同时，也是迭代器，那就无法实现这一点 —— 该对象总是会返回其自身

除此之外，当一个对象是可迭代对象的同时也是迭代器时会发生严重错误。由于__iter__会返回其自身，根据迭代器的特性，当完成一次完整的迭代后，该迭代器已经失效。此时若想重新迭代则需要使用iter()创建新的迭代器，但是使用iter()创建该对象的新迭代器时总是会返回该对象自身，而该对象自身对应的迭代器已经耗尽无法进行迭代。下述例子描述了这一问题。该例子中，第一次迭代完全正常，但是尝试进行第二次迭代时会发现迭代器为空。

In [24]:
class TestListIter:

    def __init__(self, components):
        self.components = list(components)
    
    def __iter__(self):
        """
        迭代器模式
        返回自身作为迭代器
        """
        return self
    
    def __next__(self):
        """
        迭代器要求实现的next方法
        """
        try:
            component = self.components.pop()
        except IndexError:
            raise StopIteration()
        return component
    
    def __repr__(self):
        return str(self.components)

test_list = TestListIter([1, 2, 3, 4])
print("第一次迭代")
print("所有元素：", test_list)
for i in test_list:
    print(i)

print("\n\n第二次迭代")
print("所有元素：", test_list)
for i in test_list:
    print(i)

第一次迭代
所有元素： [1, 2, 3, 4]
4
3
2
1


第二次迭代
所有元素： []


## 生成器

Python利用yield关键字定义生成器函数。只要函数中存在yield关键字，该函数就是一个生成器函数，当调用该函数时会返回一个生成器对象。使用next()函数处理生成器对象后，该生成器对象会向前执行一次yield。多次调用next()函数直至所有yield均运行完成后，会抛出StopIteration。

就工作原理而言，生成器函数显然不同于一般的函数。本书对生成器的描述如下：

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成器传递给next()时，生成器函数会向前，执行函数定义体中下一个yield语句、返回产出的值并在函数定义体的当前位置停止。最终，函数的定义体返回时，外层的生成器对象会抛出StopIteration异常 —— 这一点与迭代器协议一致。

上述描述有非常多的信息：
1. 上述描述称返回的值为“产出的值”。从原理上来看，这一描述非常准确，因为生成器并不提前存储值然后适时返回这些值，而是在运行过程中生成这些值
2. 上述对生成器的描述类似于断点调试，只不过使用next()运行生成器后会直接运行到下一个yield

生成器显然可以作为迭代器来使用，下述例子展示了如何将生成器作为迭代器。不同于TestListIter，TestListIter2显然能够正常执行迭代器的功能。值得注意的是，使用iter()调用后会返回一个生成器而不是迭代器。

In [38]:
class TestListIter2:

    def __init__(self, components):
        self.components = list(components)
    
    def __iter__(self):
        """
        返回一个生成器
        """
        for component in self.components:
            yield component
    
    def __repr__(self):
        return str(self.components)

test_list = TestListIter2([1, 2, 3, 4])
print("第一次迭代")
print("所有元素：", test_list)
print("iter(test_list)类型：", type(iter(test_list)))
for i in range(3):
    print(next(iter(test_list)))

print("\n\n第二次迭代")
print("所有元素：", test_list)
for i in test_list:
    print(i)

第一次迭代
所有元素： [1, 2, 3, 4]
iter(test_list)类型： <class 'generator'>
1
1
1


第二次迭代
所有元素： [1, 2, 3, 4]
1
2
3
4


### 生成器的“惰性”和生成器表达式

生成器具有“惰性”，其通常在需要时才真正生成值并且返回。这一工作方式显然不同于从列表或者其他序列中读值 —— 通常需要实现准备好整个序列，然后按照特定的方式获取序列中的值。

下述例子是生成器表达式和列表的对比。对于列表，首先会准备好整个序列，然后在for循环的时候逐个输出；生成器则是在for循环的时候才真正运行。

In [40]:
def gen_ABC():
    print("start")
    yield "A"
    print("continue")
    yield "B"
    print("end.")

print("\n\n列表推导")
res1 = [x*3 for x in gen_ABC()]
print(type(res1), res1)
for i in res1:
    print("--->", i)

print("\n\n生成器表达式")
res2 = (x*3 for x in gen_ABC())
print(type(res2), res2)
for i in res2:
    print("--->", i)



列表推导
start
continue
end.
<class 'list'> ['AAA', 'BBB']
---> AAA
---> BBB


生成器表达式
<class 'generator'> <generator object <genexpr> at 0x000001C749A41D48>
start
---> AAA
continue
---> BBB
end.


## 示例：斐波那契数列生成器

本书给出了一个等差数列的例子，这里给出一个斐波那契数列的例子。下述构造的斐波那契数列生成器用于返回斐波那契数列的前n项，其签名为Fibonacci(n)，n用于指明输出多少项，若没有指明n则会无限输出。

In [23]:
class ClsFibonacci:

    def __init__(self, n=None):
        self.n = n

    def __iter__(self):
        _first = 0
        _second = 1
        index = 0
        forever = self.n is None
        while forever or index < self.n:
            yield _second
            index += 1

            _temp = _second
            _second += _first
            _first = _temp


def funcFibonacci(n=None):
    _first = 0
    _second = 1
    index = 0
    forever = n is None
    while forever or index < n:
        yield _second
        index += 1

        _temp = _second
        _second += _first
        _first = _temp
            

cls_fibonacci_generator = ClsFibonacci(10)
from collections import abc
print(list(cls_fibonacci_generator))

func_fibonacci_generator = funcFibonacci(10)
print(list(cls_fibonacci_generator))

print("\n\n斐波那契数列无限生成器:")
infinity_fibonacci_generator = funcFibonacci()
for i in range(1, 11):
    print("第{}项为：{}".format(i, next(infinity_fibonacci_generator)))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
<class 'generator'>
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


斐波那契数列无限生成器:
第1项为：1
第2项为：1
第3项为：2
第4项为：3
第5项为：5
第6项为：8
第7项为：13
第8项为：21
第9项为：34
第10项为：55


## yield from

yield from是Python3.3中新增的语法，能够更简短的描述生成器。字节码清晰描述了使用yield和yield from的不同，从使用上来说，yield from更像是一种封装。

In [17]:
import dis

def generator1(_list):
    for i in _list:
        yield i

def generator2(_list):
    yield from _list

test_list = [1, 2, 3, 4]
test1 = generator1(test_list)
print(list(test1))
print(dis.dis(generator1))

print("\n\n yield from:")
test2 = generator2(test_list)
print(list(test2))
print(dis.dis(generator2))

[1, 2, 3, 4]
  4           0 SETUP_LOOP              18 (to 20)
              2 LOAD_FAST                0 (_list)
              4 GET_ITER
        >>    6 FOR_ITER                10 (to 18)
              8 STORE_FAST               1 (i)

  5          10 LOAD_FAST                1 (i)
             12 YIELD_VALUE
             14 POP_TOP
             16 JUMP_ABSOLUTE            6
        >>   18 POP_BLOCK
        >>   20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None


 yield from:
[1, 2, 3, 4]
  8           0 LOAD_FAST                0 (_list)
              2 GET_YIELD_FROM_ITER
              4 LOAD_CONST               0 (None)
              6 YIELD_FROM
              8 POP_TOP
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE
None


## iter函数的特殊用法

iter()函数可以接收两个参数，第一个为可迭代对象，第二个为哨符。在迭代时，若返回了和哨符相等的元组则抛出StopIteration终止迭代。这一用法可以代替一些循环+if判断退出的应用场景。

In [13]:
import random

random.seed(0)

def d6():
    return random.randint(1, 6)

d6_iter = iter(d6, 1)
print(type(d6_iter))
for roll in d6_iter:
    print(roll)

<class 'callable_iterator'>
4
4


## 生成器和迭代器

生成器和迭代器的异同

1. 从接口上来说
所有生成器都是迭代器，因为生成器必然会实现__iter__方法。所有的生成器对象均可以通过isinstance对abc.Iterator类型的检查
2. 从实现方式上来说
生成器在Python中有两种实现方法，其一是通过含有yield关键字的函数，其二是通过生成器表达式。无论通过哪种方式，得到的对象均是GeneratorType类型，并且均会实现符合迭代器协议的接口。因此生成器在Python中是迭代器，反过来，迭代器不一定是生成器 —— 只要通过经典的迭代器实现方式即可
3. 从概念上来说
从概念上来说，生成器和迭代器完全不同。迭代器用于遍历集合，其本质上是从数据源中读取元素，而不修改数据源。生成器则可以直接产出值。即便是基于一个集合，生成器也可以基于这个集合派生出集合外的元素。

## 总结

1. 迭代器用于从集合中取出元素；生成器用于“凭空”生成元素
2. Python从语义上集成了迭代器模式因此对迭代的支持非常好
3. 只要实现了__iter__方法或者__getitem__方法，该类的实例就是可迭代对象
4. 对于一个迭代器，其要求实现__next__方法以及__iter__方法
5. 可迭代对象不是迭代器（但是迭代器必然是可迭代对象），当一个存储数据的对象是可迭代对象同时也是迭代器时可能发生严重错误
6. Python使用yield关键字定义生成器函数，当一个函数中存在yield关键字时该函数就是一个生成器函数
7. 生成器可以作为迭代器使用，但是不同于迭代器从已知的序列中逐个输出元素，生成器在程序运行过程中“产出”元素
8. 生成器表达式不同于列表推导，通过生成器表达式得到的生成器在迭代时才会真正运行相应的代码。列表推导则在迭代前就返回了包含所有元素的完整序列
9. Python内置了大量生成器，这些生成器可以满足一些应用上的需求
10. iter()函数不仅可以接收一个参数，其也可以接受两个参数。当输入一个参数时，iter()函数返回一个迭代器。以iter(func, sentinel)形式调用时返回一个callable_iterator对象，这种形式能够使用任何函数构造迭代器。
11. 考虑到一些大型数据集并不能事先全部放入内存，仅在需要时才产出元素的生成器非常适合处理这些数据集